# Installing Java 8
Hadoop is a java programming-based data processing framework

OpenJDK is a development environment for building applications, applets, and components using the Java programming language.

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!update-alternatives --set jps /usr/lib/jvm/java-8-openjdk-amd64/bin/jps
!java -version

#Finding the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"
!apt-get install openssh-server -qq > /dev/null
!service ssh start

!grep Port /etc/ssh/sshd_config

#Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa

# See id_rsa.pub content
!more /root/.ssh/id_rsa.pub

#Copying the key to autorized keys
!cat $HOME/.ssh/id_rsa.pub > $HOME/.ssh/authorized_keys
#Changing the permissions on the key
!chmod 0600 ~/.ssh/authorized_keys

#Conneting with the local machine
!ssh -o StrictHostKeyChecking=no localhost uptime


#Downloading Hadoop 3.2.3
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz

#Untarring the file
!sudo tar -xzf hadoop-3.2.3.tar.gz
#Removing the tar file
!rm hadoop-3.2.3.tar.gz


#Copying the hadoop files to user/local
!cp -r hadoop-3.2.3/ /usr/local/
#-r copy directories recursively

#Adding JAVA_HOME directory to hadoop-env.sh file
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.3/etc/hadoop/hadoop-env.sh

import os
#Creating environment variables
#Creating Hadoop home variable

os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.3"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += f'{os.environ["JAVA_HOME"]}/bin:{os.environ["JRE_HOME"]}/bin:{os.environ["HADOOP_HOME"]}/sbin'

#Dowloading text example to use as input
# !wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)
openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64/jre/
 * Starting OpenBSD Secure Shell server sshd
   ...done.
#Port 22
#GatewayPorts no
Generating public/private rsa key pair.
/root/.ssh/id_rsa already exists.
Overwrite (y/n)? y
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:oG90c/k+U2eGpRkzaszmYJKoo8oBtvba2tX4Ohc96nE root@d959ca9afeaa
The key's randomart image is:
+---[RSA 3072]----+
|                 |
|                 |
|      .          |
|     . .   .  + .|
|..  . ..S.oo . O |
|o .  o+oo=o.* = +|
| +   +oooE.=.. + |
|o = +o.oo  .+    |
|.=+= o=o    .o   |
+----[SHA256]-----+
ssh-rsa AAAAB3

In [3]:
#Adding required property to core-site.xlm file
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>
EOF

In [4]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>

</configuration>
EOF

In [5]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.application.classpath</name>
    <value>$HADOOP_HOME/share/hadoop/mapreduce/*:$HADOOP_HOME/share/hadoop/mapreduce/lib/*</value>
  </property>

</configuration>
EOF

In [6]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
<property>
    <description>The hostname of the RM.</description>
    <name>yarn.resourcemanager.hostname</name>
    <value>localhost</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.nodemanager.env-whitelist</name>
    <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_HOME,PATH,LANG,TZ,HADOOP_MAPRED_HOME</value>
  </property>

<!-- Site specific YARN configuration properties -->

</configuration>
EOF

# Formatting the HDFS Filesystem

Before HDFS can be used for the first time the file system must be formatted. The formatting process creates an empty file system by creating the storage directories and the initial versions of the NameNodes

In [7]:
!$HADOOP_HOME/bin/hdfs namenode -format

#Creating other necessary enviroment variables before starting nodes
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

#Launching hdfs deamons
!$HADOOP_HOME/sbin/start-dfs.sh

#Launching yarn deamons
#nohup causes a process to ignore a SIGHUP signal
!nohup $HADOOP_HOME/sbin/start-yarn.sh

#Listing the running deamons
!jps

2024-05-06 17:17:30,827 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = d959ca9afeaa/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.3
STARTUP_MSG:   classpath = /usr/local/hadoop-3.2.3/etc/hadoop:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/dnsjava-2.1.7.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jersey-server-1.19.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jersey-core-1.19.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/audience-annotations-0.5.0.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-configuration2-2.1.1.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/hadoop-annotations-3.2.3.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-logging-1.1.3.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/javax.servlet-api-3.1.0.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/asm-5.0.4.jar:/usr/local/hadoop-

### Monitoring Hadoop cluster with hadoop admin commands

In [8]:
#Report the basic file system information and statistics
!$HADOOP_HOME/bin/hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 85080453120 (79.24 GB)
DFS Remaining: 85080428544 (79.24 GB)
DFS Used: 24576 (24 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: d959ca9afeaa
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 30560960512 (28.46 GB)
DFS Remaining: 85080428544 (79.24 GB)
DFS Used%: 0.00%
DFS Remaining%: 73.56%
Configured Cache Capacity: 0 (0 B)
Cache

# MapReduce

In [9]:
#Dowloading text example to use as input (if it has not been donwloaded yet)
!wget -q https://www.gutenberg.org/cache/epub/15038/pg15038.txt

#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count
#Coping file from local file system to HDFS
!$HADOOP_HOME/bin/hdfs dfs -put pg15038.txt /word_count

#Exploring Hadoop folder
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count

# Run MapReduce Example using JAVA
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar wordcount /word_count/pg15038.txt /word_count/output/

Found 1 items
-rw-r--r--   1 root supergroup      70605 2024-05-06 17:18 /word_count/pg15038.txt
2024-05-06 17:18:33,108 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-06 17:18:33,837 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1715015886393_0001
2024-05-06 17:18:34,201 INFO input.FileInputFormat: Total input files to process : 1
2024-05-06 17:18:34,326 INFO mapreduce.JobSubmitter: number of splits:1
2024-05-06 17:18:35,003 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1715015886393_0001
2024-05-06 17:18:35,005 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-05-06 17:18:35,273 INFO conf.Configuration: resource-types.xml not found
2024-05-06 17:18:35,274 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-05-06 17:18:35,724 INFO impl.YarnClientImpl: Submitted application application_1715015886393_0001
2024-05-06 17:18:35,826 INFO mapr

In [10]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-05-06 17:19 /word_count/output/_SUCCESS
-rw-r--r--   1 root supergroup      38103 2024-05-06 17:19 /word_count/output/part-r-00000


In [11]:
#Printing out first 100 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count/output/part-r-00000 | head -100

"Gevecht	1
#15038]	1
$5,000)	1
's	4
($1	1
(801)	1
(Borneo),	1
(Sumatra)	1
(a)	1
(and	1
(any	1
(b)	1
(c)	1
(does	1
(if	1
(negentienhonderd	1
(op	1
(or	3
(trademark/copyright)	1
(www.gutenberg.org),	1
(“the	1
***	4
-	3
1.	1
1.A.	1
1.B.	1
1.C	1
1.C.	1
1.D.	1
1.E	1
1.E.	1
1.E.1	3
1.E.1.	2
1.E.2.	1
1.E.3.	1
1.E.4.	1
1.E.5.	1
1.E.6.	1
1.E.7	2
1.E.7.	1
1.E.8	2
1.E.8.	2
1.E.9.	3
1.F.	1
1.F.1.	1
1.F.2.	1
1.F.3,	3
1.F.3.	2
1.F.4.	1
1.F.5.	1
1.F.6.	1
10	1
103°6'	1
11	1
14	1
14,	2
1500	1
1680.	1
18	1
1868,	1
1877;	1
1883	3
1883,	1
1884	1
2.	1
20%	1
2001,	1
2005	1
2020	1
21	1
22	1
23	1
24	1
25	1
26	3
27	10
27,	1
27;	1
28	1
28,	2
3	2
3.	1
30	1
4	1
4,	1
4.	1
5.	1
50	1
501(c)(3)	1
596-1887.	1
60	1
64-6221541.	1
6°7'	1
809	1
84116,	1
90	2
ACTUAL,	1
AGREE	2
AGREEMENT	1
AND	1


# Hadoop Streaming Using Python

Hadoop Streaming is a feature that comes with Hadoop and allows users or developers to use various different languages for writing MapReduce programs like Python, C++, Ruby, etc.

The utility will create a Map/Reduce job, submit the job to an appropriate cluster, and monitor the progress of the job until it completes.

In [12]:
#Exploring Hadoop utilities available
!ls $HADOOP_HOME/share/hadoop/tools/lib/

aliyun-java-sdk-core-4.5.10.jar      hadoop-gridmix-3.2.3.jar
aliyun-java-sdk-kms-2.11.0.jar	     hadoop-kafka-3.2.3.jar
aliyun-java-sdk-ram-3.1.0.jar	     hadoop-openstack-3.2.3.jar
aliyun-sdk-oss-3.13.0.jar	     hadoop-resourceestimator-3.2.3.jar
aws-java-sdk-bundle-1.11.901.jar     hadoop-rumen-3.2.3.jar
azure-data-lake-store-sdk-2.2.9.jar  hadoop-sls-3.2.3.jar
azure-keyvault-core-1.0.0.jar	     hadoop-streaming-3.2.3.jar
azure-storage-7.0.0.jar		     ini4j-0.5.4.jar
hadoop-aliyun-3.2.3.jar		     jdom2-2.0.6.jar
hadoop-archive-logs-3.2.3.jar	     kafka-clients-2.8.1.jar
hadoop-archives-3.2.3.jar	     lz4-java-1.7.1.jar
hadoop-aws-3.2.3.jar		     ojalgo-43.0.jar
hadoop-azure-3.2.3.jar		     opentracing-api-0.33.0.jar
hadoop-azure-datalake-3.2.3.jar      opentracing-noop-0.33.0.jar
hadoop-datajoin-3.2.3.jar	     opentracing-util-0.33.0.jar
hadoop-distcp-3.2.3.jar		     org.jacoco.agent-0.8.5-runtime.jar
hadoop-extras-3.2.3.jar		     wildfly-openssl-1.0.7.Final.jar
hadoop-fs2img-3.2.3.

In [13]:
#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count_with_python

In [14]:
#Copying the file from local file system to Hadoop distributed file system (HDFS)
!$HADOOP_HOME/bin/hdfs dfs -put /content/pg15038.txt /word_count_with_python

## Mapper

The mapper is an executable that reads all input records from a file/s and generates an output in the form of key-value pairs which works as input for the Reducer.

In [15]:
%%writefile mapper.py

#!/usr/bin/env python

#'#!' is known as shebang and used for interpreting the script

# import sys because we need to read and write data to STDIN and STDOUT
import sys

# reading entire line from STDIN (standard input)
for line in sys.stdin:
  # to remove leading and trailing whitespace
  line = line.strip()
  # split the line into words, output data type list
  words = line.split()

  # we are looping over the words array and printing the word
  # with the count of 1 to the STDOUT
  for word in words:
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s\t%s' % (word, 1))

Writing mapper.py


## Reducer

The reducer is an executable that reads all the intermediate key-value pairs generated by the mapper and generates a final output as a result of a computation operation like addition, filtration, and aggregation.

Both the mapper and the reducer read the input from stdin (line by line) and emit the output to stdout.

In [16]:
%%writefile reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# read the entire line from STDIN
for line in sys.stdin:
  # remove leading and trailing whitespace
  line = line.strip()
  # splitting the data on the basis of tab we have provided in mapper.py
  word, count = line.split('\t', 1)
  # convert count (currently a string) to int
  try:
    count = int(count)
  except ValueError:
    # count was not a number, so silently
    # ignore/discard this line
    continue

  # this IF-switch only works because Hadoop sorts map output
  # by key (here: word) before it is passed to the reducer
  if current_word == word:
    current_count += count
  else:
    if current_word: #to not print current_word=None
      # write result to STDOUT
      print('%s\t%s' % (current_word, current_count))
    current_count = count
    current_word = word

# do not forget to output the last word if needed!
if current_word == word:
  print('%s\t%s' % (current_word, current_count))

Writing reducer.py


In [17]:
#Testing our MapReduce job locally (Hadoop does not participate here)
!cat pg15038.txt | python mapper.py | sort -k1,1 | python reducer.py | head -50
#We apply sorting after the mapper because it is the default operation in MapReduce architecture

***	4
-	3
•	4
($1	1
$5,000)	1
1.	1
10	1
103°6'	1
11	1
14	1
14,	2
1500	1
#15038]	1
1680.	1
18	1
1868,	1
1877;	1
1883	3
1883,	1
1884	1
1.A.	1
1.B.	1
1.C	1
1.C.	1
1.D.	1
1.E	1
1.E.	1
1.E.1	3
1.E.1.	2
1.E.2.	1
1.E.3.	1
1.E.4.	1
1.E.5.	1
1.E.6.	1
1.E.7	2
1.E.7.	1
1.E.8	2
1.E.8.	2
1.E.9.	3
1.F.	1
1.F.1.	1
1.F.2.	1
1.F.3,	3
1.F.3.	2
1.F.4.	1
1.F.5.	1
1.F.6.	1
2.	1
20%	1
2001,	1
Traceback (most recent call last):
  File "/content/reducer.py", line 32, in <module>
    print('%s\t%s' % (current_word, current_count))
BrokenPipeError: [Errno 32] Broken pipe


In [18]:
#Changing the permissions of the files
!chmod 777 /content/mapper.py /content/reducer.py
#Setting 777 permissions to a file or directory means that it will be readable, writable and executable by all users

In [19]:
#Running MapReduce programs
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.3.jar \
  -input /word_count_with_python/pg15038.txt \
  -output /word_count_with_python/output \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

packageJobJar: [/tmp/hadoop-unjar6313933521514989150/] [] /tmp/streamjob2922933088969711253.jar tmpDir=null
2024-05-06 17:20:07,168 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-06 17:20:07,508 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-06 17:20:07,911 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1715015886393_0002
2024-05-06 17:20:08,279 INFO mapred.FileInputFormat: Total input files to process : 1
2024-05-06 17:20:08,777 INFO mapreduce.JobSubmitter: number of splits:2
2024-05-06 17:20:09,034 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1715015886393_0002
2024-05-06 17:20:09,036 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-05-06 17:20:09,327 INFO conf.Configuration: resource-types.xml not found
2024-05-06 17:20:09,328 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-05-06 17:20:09,4

In [20]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-05-06 17:20 /word_count_with_python/output/_SUCCESS
-rw-r--r--   1 root supergroup      38103 2024-05-06 17:20 /word_count_with_python/output/part-00000


In [21]:
#Printing out first 100 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count_with_python/output/part-00000 | head -100

"Gevecht	1
#15038]	1
$5,000)	1
's	4
($1	1
(801)	1
(Borneo),	1
(Sumatra)	1
(a)	1
(and	1
(any	1
(b)	1
(c)	1
(does	1
(if	1
(negentienhonderd	1
(op	1
(or	3
(trademark/copyright)	1
(www.gutenberg.org),	1
(“the	1
***	4
-	3
1.	1
1.A.	1
1.B.	1
1.C	1
1.C.	1
1.D.	1
1.E	1
1.E.	1
1.E.1	3
1.E.1.	2
1.E.2.	1
1.E.3.	1
1.E.4.	1
1.E.5.	1
1.E.6.	1
1.E.7	2
1.E.7.	1
1.E.8	2
1.E.8.	2
1.E.9.	3
1.F.	1
1.F.1.	1
1.F.2.	1
1.F.3,	3
1.F.3.	2
1.F.4.	1
1.F.5.	1
1.F.6.	1
10	1
103°6'	1
11	1
14	1
14,	2
1500	1
1680.	1
18	1
1868,	1
1877;	1
1883	3
1883,	1
1884	1
2.	1
20%	1
2001,	1
2005	1
2020	1
21	1
22	1
23	1
24	1
25	1
26	3
27	10
27,	1
27;	1
28	1
28,	2
3	2
3.	1
30	1
4	1
4,	1
4.	1
5.	1
50	1
501(c)(3)	1
596-1887.	1
60	1
64-6221541.	1
6°7'	1
809	1
84116,	1
90	2
ACTUAL,	1
AGREE	2
AGREEMENT	1
AND	1


In [25]:
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /word_count_with_python/output/part-00000 /content/hdfs-wordcount.txt

copyToLocal: `/content/hdfs-wordcount.txt': File exists
